In [1]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

warnings.filterwarnings("ignore")

### Pre-processing

In [2]:
df_110 = pd.read_csv(("../Resources/110justice.csv"))
df_110.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
1911,110,2017,2,0,1,2,55
1912,110,2017,1,0,9,9,0
1913,110,2017,2,0,2,2,0
1914,110,2017,2,0,2,6,0
1915,110,2017,2,0,1,6,0


In [3]:
target_110 = df_110["direction"]
target_names = ["conservative", "liberal"]

In [4]:
data_110 = df_110.drop(["direction", "justice"], axis=1)
data_110.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,1994,0,2,4,0
1,1994,0,2,6,0
2,1994,0,1,6,0
3,1994,0,9,0,0
4,1994,0,8,2,0


In [5]:
# dummy (One-Hot) encoding
term_df = pd.get_dummies(data_110["term"], prefix="term")
precAlt_df = pd.get_dummies(data_110["precedentAlteration"], prefix="precedentAlteration")
issueArea_df = pd.get_dummies(data_110["issueArea"], prefix="issueArea")
lawType_df = pd.get_dummies(data_110["lawType"], prefix="lawType")
caseOriginState_df = pd.get_dummies(data_110["caseOriginState"], prefix="caseOriginState")

In [6]:
# concatenate encoded features into mega-dataframe
oneHot_features = pd.concat([term_df, precAlt_df], axis=1)
oneHot_features = pd.concat([oneHot_features, issueArea_df], axis=1)
oneHot_features = pd.concat([oneHot_features, lawType_df], axis=1)
oneHot_features = pd.concat([oneHot_features, caseOriginState_df], axis=1)
oneHot_features.head()

,term_1994,term_1995,term_1996,term_1997,term_1998,term_1999,term_2000,term_2001,term_2002,term_2003,...,caseOriginState_49,caseOriginState_50,caseOriginState_51,caseOriginState_52,caseOriginState_53,caseOriginState_55,caseOriginState_56,caseOriginState_57,caseOriginState_58,caseOriginState_59
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = oneHot_features
feature_names = oneHot_features.columns
y = target_110

### Evaluating different classifiers

In [8]:
clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier()
clf_nb = GaussianNB()
clf_svm = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

5-fold cross validation:

F1: 0.325 [Logistic Regression]
F1: 0.395 [Random Forest]
F1: 0.232 [Naive Bayes]
F1: 0.279 [Linear SVM]


In [9]:
# using all but naive bayes (lowest score) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [10]:
v_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [11]:
new_labels = ["Logistic Regression", "Random Forest", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_lr, clf_rf, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.325 [Logistic Regression]
F1: 0.425 [Random Forest]
F1: 0.279 [Linear SVM]
F1: 0.298 [Voting_Classifier_Hard]
F1: 0.325 [Voting_Classifier_Soft]


### SVM Model Training

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.600
Test Acc: 0.576


In [19]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.45      0.12      0.19       159
     liberal       0.59      0.90      0.71       225

   micro avg       0.58      0.58      0.58       384
   macro avg       0.52      0.51      0.45       384
weighted avg       0.53      0.58      0.50       384



In [20]:
print(confusion_matrix(predictions, y_test))
predictions[:15], y_test[:15].ravel()

[[ 19  23]
 [140 202]]


(array([2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2], dtype=int64))

### Save Model and Confirm Load

In [22]:
import pickle

filename = '110_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [23]:
#Load Model from Disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.5755208333333334
